In [ ]:
# syft absolute
import syft as sy

In [ ]:
high_client = sy.login(
    url="http://localhost:8080",
    email="data_scientist@openmined.org",
    password="verysecurepassword",
)

In [ ]:
# If this cell fails, make sure you have the secrets in the .env file
requested_keys = [ 
    "dataset_2",
    "table_1",
    "table_2",
    "table_2_col_id",
    "table_2_col_score",
]
secrets = sy.get_nb_secrets(requested_keys = requested_keys)

In [ ]:
# # add your values to secrets.json in this folder
# secrets = sy.get_nb_secrets(
#     {
#         "dataset_2": "dataset2",
#         "table_1": "table1",
#         "table_2": "table2",
#         "table_2_col_id": "id",
#         "table_2_col_score": "score",
#     }
# )

In [ ]:
high_client.custom_api.api_endpoints()

In [ ]:
high_client.api.services.bigquery.test_query

In [ ]:
FUNC_NAME = "popular"
QUERY = f"SELECT {secrets['table_2_col_id']}, AVG({secrets['table_2_col_score']}) AS average_score \
    FROM {secrets['dataset_2']}.{secrets['table_2']} \
    GROUP BY {secrets['table_2_col_id']} \
    LIMIT 10000"

high_client.api.services.bigquery.test_query(sql_query=QUERY)

In [ ]:
res1 = high_client.api.services.bigquery.submit_query(func_name=FUNC_NAME, query=QUERY)
res1

In [ ]:
def extract_code_path(response):
    # stdlib
    import re

    pattern = r"client\.code\.(\w+)\(\)"
    match = re.search(pattern, str(response))
    if match:
        extracted_code = match.group(1)
        return extracted_code
    return None

In [ ]:
func_name = extract_code_path(res1)
print(func_name)
api_method = getattr(high_client.code, func_name, None)
api_method

In [ ]:
api_method()

In [ ]:
FUNC_NAME = "large_sample"
LARGE_SAMPLE_QUERY = (
    f"SELECT * FROM {secrets['dataset_2']}.{secrets['table_2']} LIMIT 1000000"
)

In [ ]:
res2 = high_client.api.services.bigquery.submit_query(
    func_name=FUNC_NAME, query=LARGE_SAMPLE_QUERY
)

In [ ]:
func_name = extract_code_path(res2)
print(func_name)
api_method_2 = getattr(high_client.code, func_name, None)
api_method_2

In [ ]:
api_method_2()